In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import ADASYN
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [2]:
df = pd.read_csv("D:\\project cuối khóa SIC\\diabetes.csv")

In [3]:
#thay giá trị 0 của (glucose, bloodpressure, skinthickness, BMI, insulin) sang NaN  
replaced_columns = ['Glucose', 'BloodPressure','SkinThickness','BMI','Insulin']
df[replaced_columns] = df[replaced_columns].replace(0, np.nan)

In [4]:
#Xử lý outlier

from scipy import stats

#Z_score
def remove_outliers_zscore(df, columns, threshold=3):
    df_cleaned = df.copy()
    for col in columns:
        # Tính z-score cho các giá trị không null
        z_scores = stats.zscore(df_cleaned[col].dropna())
        abs_z_scores = np.abs(z_scores)
        # Xác định outliers
        upper_outliers = z_scores > threshold
        lower_outliers = z_scores < -threshold
        non_outliers = (abs_z_scores <= threshold)  # Không phải outliers
        # Lấy giá trị max và min của các giá trị không phải outliers
        max_non_outlier_value = df_cleaned[col].dropna()[non_outliers].max()
        min_non_outlier_value = df_cleaned[col].dropna()[non_outliers].min()
        # Thay thế outliers ở khoảng trên bằng max_non_outlier_value
        upper_outliers_indices = df_cleaned[col].dropna().index[upper_outliers]
        df_cleaned.loc[upper_outliers_indices, col] = max_non_outlier_value       
        # Thay thế outliers ở khoảng dưới bằng min_non_outlier_value
        lower_outliers_indices = df_cleaned[col].dropna().index[lower_outliers]
        df_cleaned.loc[lower_outliers_indices, col] = min_non_outlier_value
    return df_cleaned

columns_zscore = ['Glucose', 'BloodPressure', 'SkinThickness', 'BMI']
df = remove_outliers_zscore(df, columns_zscore, threshold=3)

#IQR
def remove_outliers_iqr(df, columns, factor=1.5):
    df_cleaned = df.copy()
    for col in columns:
        Q1 = df_cleaned[col].quantile(0.25)
        Q3 = df_cleaned[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - (IQR * factor)
        upper_bound = Q3 + (IQR * factor)  
        # Xác định các giá trị không phải outliers
        non_outliers = (df_cleaned[col] >= lower_bound) & (df_cleaned[col] <= upper_bound)
        # Lấy giá trị max của các giá trị không phải outliers
        max_non_outlier_value = df_cleaned[col].dropna()[non_outliers].max()
        # Xác định outliers và thay thế bằng max_non_outlier_value
        outliers = (df_cleaned[col] < lower_bound) | (df_cleaned[col] > upper_bound)
        df_cleaned.loc[outliers, col] = max_non_outlier_value
    return df_cleaned

columns_iqr = ['Pregnancies', 'Insulin', 'DiabetesPedigreeFunction', 'Age']
df = remove_outliers_iqr(df, columns_iqr, factor=1.5)

In [5]:
#split x(input) và y(output)
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [6]:
# Điền giá trị trung bình cho cột glucose (cột 1)
imputer_mean = SimpleImputer(missing_values=np.nan, strategy="mean")
X[:, 1] = imputer_mean.fit_transform(X[:, [1]]).ravel()

#Xóa hàng null trong columns BMI 
mask = ~np.isnan(X[:, 5])  # Tạo mask để kiểm tra hàng nào có giá trị null trong cột BMI (cột 5)
      # Áp dụng mask lên cả X và y để đảm bảo đồng bộ
X = X[mask]
y = y[mask]

# Khởi tạo KNNImputer cho các cột Bloodpressure (2), Skinthickness (3), và Insulin (4)
knn_imputer = KNNImputer(n_neighbors=5)
X[:, [2, 3, 4]] = knn_imputer.fit_transform(X[:, [2, 3, 4]])

In [7]:
#Split dataset thành train_set(85%) và test_set(15%)
np.random.seed(101)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

In [8]:
#Chuẩn hóa dữ liệu (feature scaling)
sc = StandardScaler()
X_train[:, [0, 1, 2, 3, 4, 5, 6, 7]] = sc.fit_transform(X_train[:, [0, 1, 2, 3, 4, 5, 6, 7]])
X_test[:, [0, 1, 2, 3, 4, 5, 6, 7]] = sc.transform(X_test[:, [0, 1, 2, 3, 4, 5, 6, 7]])

In [9]:
#Cân bằng data
# Khởi tạo ADASYN với tỷ lệ sampling_strategy (cân bằng data)
adasyn = ADASYN(sampling_strategy='auto', n_neighbors=5, random_state=101)
# Áp dụng ADASYN để tạo thêm các mẫu tổng hợp từ tập huấn luyện
X_train_resampled, y_train_resampled = adasyn.fit_resample(X_train, y_train)

In [10]:
# Khởi tạo mô hình Logistic Regression
logreg = LogisticRegression(random_state=101, max_iter=1000)

In [11]:
# Đưa data đã xử lý vào model
logreg.fit(X_train_resampled, y_train_resampled)

LogisticRegression(max_iter=1000, random_state=101)

In [12]:
import joblib

# Giả sử bạn đã có một mô hình đã được huấn luyện
# logreg = LogisticRegression(random_state=101, max_iter=1000)
# logreg.fit(X_train_resampled, y_train_resampled)
# Lưu mô hình
joblib.dump(logreg, 'logreg_model.pkl')
# Lưu scaler nếu bạn sử dụng chuẩn hóa
joblib.dump(sc, 'scaler.pkl')

['scaler.pkl']

In [13]:
# Tải mô hình và scaler đã lưu
logreg = joblib.load('logreg_model.pkl')
sc = joblib.load('scaler.pkl')

# Nhập chỉ số bệnh nhân mới và dự đoán
print("\n________________________Hãy nhập các chỉ số của bệnh nhân ở bên dưới:_______________________")
pregnancies = input("Số lần mang thai: ")
glucose = input("Chỉ số đường huyết (OGTT): ")
blood_pressure = input("Huyết áp tâm trương (mmHg): ")
skin_thickness = input("Độ dày nếp gấp da ở cơ tam đầu (mm):  ")
insulin = input("Nồng độ Insulin (μU/mL): ")
bmi = input("Chỉ số BMI (kg/m2): ")
diabetes_pedigree_function = input("Diabetes Pedigree Function: ")
age = input("Tuổi: ")

print("\n____________________________________Các chỉ số đã nhập:______________________________________")
print(f"Số lần mang thai: {pregnancies}")
print(f"Chỉ số đường huyết (OGTT): {glucose}")
print(f"Huyết áp tâm trương (mmHg): {blood_pressure}")
print(f"Độ dày nếp gấp da ở cơ tam đầu (mm): {skin_thickness}")
print(f"Nồng độ Insulin (μU/mL): {insulin}")
print(f"Chỉ số BMI (kg/m2): {bmi}")
print(f"Diabetes Pedigree Function: {diabetes_pedigree_function}")
print(f"Tuổi: {age}")

# Tạo mảng numpy từ dữ liệu nhập vào
new_data = np.array([[int(pregnancies), float(glucose), float(blood_pressure), float(skin_thickness), 
                      float(insulin), float(bmi), float(diabetes_pedigree_function), int(age)]])

# Chuẩn hóa dữ liệu mới
new_data_scaled = sc.transform(new_data)

# Dự đoán
predicted_value = logreg.predict(new_data_scaled)
print("\n_____________________________________________________________________________________________")
print(f"\nDự đoán tình trạng bệnh tiểu đường (1(dương tính), 0(âm tính): {predicted_value[0]}")


________________________Hãy nhập các chỉ số của bệnh nhân ở bên dưới:_______________________


Số lần mang thai:  1
Chỉ số đường huyết (OGTT):  89
Huyết áp tâm trương (mmHg):  66
Độ dày nếp gấp da ở cơ tam đầu (mm):   23
Nồng độ Insulin (μU/mL):  94
Chỉ số BMI (kg/m2):  28.1
Diabetes Pedigree Function:  0.167
Tuổi:  21



____________________________________Các chỉ số đã nhập:______________________________________
Số lần mang thai: 1
Chỉ số đường huyết (OGTT): 89
Huyết áp tâm trương (mmHg): 66
Độ dày nếp gấp da ở cơ tam đầu (mm): 23
Nồng độ Insulin (μU/mL): 94
Chỉ số BMI (kg/m2): 28.1
Diabetes Pedigree Function: 0.167
Tuổi: 21

_____________________________________________________________________________________________

Dự đoán tình trạng bệnh tiểu đường (1(dương tính), 0(âm tính): 0
